In [1]:
from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 3000, clientId=9) # For nse - Kavi

import pandas as pd

In [9]:
%%time
fspath = '../data/nse/'

df1 = pd.read_csv(fspath+'checked_old.csv')

existing_symbol = [c.contract.symbol for c in ib.positions()]

df2 = df1[~df1.symbol.isin(existing_symbol)].reset_index(drop=True)

orders = [Order(action='SELL', totalQuantity=lot, orderType='MKT') for lot in df2.lot]
contracts = [eval(c) for c in df2.option]

tickers = ib.reqTickers(*contracts)

price_dict = {t.contract.conId: t.marketPrice() for t in tickers}

Started to throttle requests
Stopped to throttle requests


Wall time: 14.1 s


In [16]:
df3 = df2.assign(conId=[c.conId for c in contracts])

In [20]:
df4 = df3.set_index('conId')

In [23]:
df4.price = df4.price.map(price_dict).fillna(df4.price)

In [28]:
df = df4.reset_index()

In [29]:
df.to_csv(fspath+'check.csv', index=None, header=True)

In [33]:
max_nlvp = 0.8    # max allowable nlv to prevent overall portfolio risk. 0.8 means 80% of NLV.
                  # max available funds for option trades = max_nlvp * NLV - initMargin

ac = ib.accountValues()
df_a = util.df(ac)

#... set max margin per position
net_liq = float(df_a[df_a.tag == 'NetLiquidation'].iloc[0].value) 
av_funds = float(df_a[df_a.tag == 'FullAvailableFunds'].iloc[0].value)

df_final = pd.read_csv(fspath+'checked.csv') # picks up the checked and ready-to-go contracts
cs = [eval(c) for c in df_final.option]  # convert the "quoted strings" from csv back to object
orders = [LimitOrder(action='SELL', totalQuantity=lot, lmtPrice=expPrice) for lot, expPrice in zip(df_final.lot, df_final.expPrice)]
print('{:d} contracts from {:d} scrips, consuming {:,.0f} margin from full available funds of {:,.0f}, giving INR {:,.0f}'.format(len(cs), \
      len(df_final.symbol.unique()), sum(df_final.margin*df_final.qty), av_funds*max_nlvp, sum(df_final.expPrice*df_final.lot*df_final.qty)))

124 contracts from 58 scrips, consuming 43,043,455 margin from full available funds of 15,145,559, giving INR 799,003
